# Importing data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os

In [ ]:
your_directory = '/Users/miguel'
raw_data_directory = your_directory + '/raw_data'
raw_data_directory

In [ ]:
# lo del df_name no tiene sentido si luego no lo puedo llamar

In [ ]:
# input raw data from different years (2010-2021)
initial_year = 2010
last_year = 2021
next_year = initial_year + 1
num_years = last_year - initial_year + 1
list_of_dfs = []
list_of_directories = []
# format example: df_2010_2011 = pd.read_csv(raw_data_directory + '/Notas de corte de Medicina 2010_2011 - Hoja 1.csv', decimal = ',')

for i in range(num_years):
    df_name = 'df' + '_' + str(initial_year) + '_' + str(next_year)
    directory = raw_data_directory + '/Notas de corte de Medicina ' + str(initial_year) + '_' + str(next_year) + ' - Hoja 1.csv'
    list_of_dfs.append(df_name)
    list_of_directories.append(directory)
    
    list_of_dfs[i] = pd.read_csv(list_of_directories[i], decimal = ',')
    
    initial_year += 1
    next_year += 1

In [ ]:
# before appending all dfs, we create a column with the year, to keep track of which df the data is coming from
initial_year = 2010

for i in range(num_years):
    list_of_dfs[i]['year'] = initial_year + i

In [ ]:
dfs_unified = pd.DataFrame()

In [ ]:
for i in range(num_years):
    dfs_unified = dfs_unified.append(list_of_dfs[i])

In [ ]:
# check the append has been done correctly: check the number of rows is the sum of all of the dfs

total_number_of_rows = 0

for i in range(num_years):
    total_number_of_rows = total_number_of_rows + len(list_of_dfs[i])

if total_number_of_rows != dfs_unified.shape[0]:
    sys.exit()
else:
    print('append has been done correctly')

In [ ]:
dfs_unified.describe()

# Data Cleaning

## Check for duplicates

In [ ]:
if dfs_unified[dfs_unified.duplicated() == True].shape[0] != 0:
    dfs_unified.drop_duplicates()
else:
    print('There are no duplicates')

## Create new unique columns out of data that we already had

In [ ]:
def drop_columns_containing_text(df, text):
    '''
    drop columns that contain a certain string/text
    '''
    df = df[df.columns.drop(list(df.filter(regex=text)))]
    return df

In [ ]:
dfs_unified = drop_columns_containing_text(dfs_unified, 'Sobre')

In [ ]:
# to keep track of raw columns, to afterwards drop them
dfs_unified_raw_columns = dfs_unified.columns.tolist()
# 'year' was created before and is not a raw column
dfs_unified_raw_columns.remove('year')
#dfs_unified_columns

In [ ]:
# tiene sentido copiar solo not null data y no toda????????

In [ ]:
def copy_data(new_col, old_col):
    '''
    copy not null data from one column to another
    '''
    dfs_unified[new_col] = np.where(dfs_unified[old_col].notnull(), dfs_unified[old_col], dfs_unified[new_col])

In [ ]:
dfs_unified['university'] = ''
copy_data('university', 'Notas de corte de Medicina')
copy_data('university', 'Universidad')
copy_data('university', 'Facultad de Medicina')

#dfs_unified['date_last_admission'] = ''
#copy_data('date_last_admission', 'Fecha de última admisión')
#copy_data('date_last_admission', 'Última fecha de admisión \n(Fecha de la nota final)')
dfs_unified['1_list'] = ''
copy_data('1_list', '1ª Lista')

dfs_unified['final_grade'] = ''
copy_data('final_grade', 'Nota de corte final de Medicina')
copy_data('final_grade', 'Nota de corte final')
copy_data('final_grade', 'Nota de corte actual')



#copy_data('19_list', '19ª lista')
#copy_data('20_list', '20ª lista')

In [ ]:
# drop all old columns
dfs_unified = dfs_unified.drop(dfs_unified_raw_columns, axis=1)

# reset index
#dfs_unified = dfs_unified.reset_index()

In [ ]:
dfs_unified[~dfs_unified['university'].str.contains('U', na=False)]

In [ ]:
dfs_unified = dfs_unified[dfs_unified['university'].str.contains('U', na=False)]

In [ ]:
dfs_unified = dfs_unified[~dfs_unified['university'].str.contains('Sobre la Universidad de Santiago:', na=False)]

In [ ]:
dfs_unified.columns

In [ ]:
dfs_unified_columns = dfs_unified.columns.tolist()[1:]

num_columns = len(dfs_unified_columns)

for i in range(num_columns):
    dfs_unified[dfs_unified_columns[i]].replace({'': np.nan}, inplace=True)

## Check for nulls

In [ ]:
# quality check: nulls

nulls_1_list = dfs_unified[dfs_unified['1_list'].isnull()].shape[0]
nulls_final_grade = dfs_unified[dfs_unified['final_grade'].isnull()].shape[0]
nulls_university = dfs_unified[dfs_unified['university'].isnull()].shape[0]

if nulls_1_list or nulls_final_grade or nulls_university != 0:
    sys.exit()
else:
    print('column 1_list has', nulls_1_list, 'null rows')
    print('column final_grade has', nulls_final_grade, 'null rows')
    print('column university has', nulls_university, 'null rows')

## Create consistent University names across years

In [ ]:
dfs_unified['university'] = dfs_unified['university'].str.lower()

In [ ]:
last_year = 2021
dfs_unified[dfs_unified['year'] == last_year]['university'].values

In [ ]:
list_of_uni_names_2021 = dfs_unified[dfs_unified['year'] == last_year]['university'].tolist()
num_of_uni_names_2021 = len(list_of_uni_names_2021)
print('There are', num_of_uni_names_2021, 'universities in 2021')

dfs_unified_list_uni = list(dfs_unified['university'].unique())
num_of_uni_names_dataset = len(dfs_unified_list_uni)
print('There are a total of', num_of_uni_names_dataset, 'universities in our dataset')

diff_uni_names = num_of_uni_names_dataset - num_of_uni_names_2021
print('This means there are a total of', diff_uni_names, 'universities that have a different name than the ones in our 2021 data, and need to be mapped')

In [ ]:
# we create a list with the universities that have different name

list_not_matched = []

for i in dfs_unified_list_uni:
    if i not in list_of_uni_names_2021:
        list_not_matched.append(i)
    else:
        continue


if len(list_not_matched) != diff_uni_names:
    sys.exit()
else:
    print('list of not matched created successfully')

list_not_matched

### Sequence Matcher

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
df1 = pd.DataFrame({
    "not_matched_uni": list_not_matched
})

df2 = pd.DataFrame({
    "list_of_uni_names_2021": list_of_uni_names_2021
})

df3 = pd.MultiIndex.from_product(
    [df1["not_matched_uni"], df2["list_of_uni_names_2021"]], names=["not_matched_uni", "list_of_uni_names_2021"]
).to_frame(index=False)

In [ ]:
df3['ratio'] = ''

for i in range(len(df3['not_matched_uni'])):
    df3['ratio'][i] = similar(df3['not_matched_uni'][i], df3['list_of_uni_names_2021'][i])

df3['not_matched_uni'].nunique()

In [ ]:
df4 = df3.groupby('not_matched_uni')['ratio'].max().reset_index()

df5 = pd.merge(df4,df3,on='not_matched_uni',how='left')
df5 = df5[df5['ratio_x'] == df5['ratio_y']]
df5

In [ ]:
# hacerlo con str.contains, crear funcion

In [ ]:
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de cantabria (santander)'), 'u. de cantabria (santander)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de extremadura (badajoz)'), 'u. de extremadura (badajoz)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de jaume i'), 'universidad jaume i', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de zaragoza - campus de huesca'), 'u. de zaragoza (campus huesca)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de santiago de compostela'), 'u. de santiago de compostela', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de zaragoza - campus de zaragoza'), 'u. de zaragoza (campus zaragoza)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad miguel hernández (san juan de alicante)'), 'u. miguel hernández (s. juan de alicante)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad pompeu fabra (barcelona)'), 'u. pompeu fabra (barcelona)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de santiago de compostela'), 'u. de santiago de compostela', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad rey juan carlos - campus de alcorcón'), 'urjc - campus alcorcón', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad autónoma de madrid'), 'u. autónoma de madrid', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de barcelona'), 'universidad de barcelona-clínico', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de clm- campus de albacete'), 'u. de c-la mancha (campus albacete)', dfs_unified['university'])
dfs_unified['university'] = np.where((dfs_unified['university'] == 'universidad de clm- campus de ciudad real'), 'u. de c-la mancha (campus c.real)', dfs_unified['university'])

In [ ]:
# quality check: make sure all university names are mapped with 2021 names

dfs_unified_list_uni = list(dfs_unified['university'].unique())

list_not_matched = []

for i in dfs_unified_list_uni:
    if i not in list_of_uni_names_2021:
        list_not_matched.append(i)
    else:
        continue

# checking length of not matched
if len(list_not_matched) != 0:
    sys.exit()
else:
    print('all university names are consistent with 2021')

# another way of checking this could be by analyzing the overall length and unique names
if dfs_unified['university'].nunique() != len(list_of_uni_names_2021):
    sys.exit()
else:
    print('all university names are consistent with 2021')

### Check if all of the universities have been delivering Medicine career since 2010

In [ ]:
# check how many universities there are per year
uni_year_grouped = dfs_unified.groupby(['year'])['university'].count().reset_index()

# style
plt.style.use('ggplot')

fig,ax=plt.subplots(figsize=(12,8))
bars = ax.bar(uni_year_grouped['year'], uni_year_grouped['university'])
# show number on top of bars
ax.bar_label(bars)

# title and axis
plt.xticks(range(min(uni_year_grouped['year']), max(uni_year_grouped['year']) + 1))
plt.yticks(range(min(uni_year_grouped['university']), max(uni_year_grouped['university']) + 1))
plt.axis([2009, 2022, 30, 36])
plt.title('Number of universities per year')
plt.xlabel('Year')
plt.ylabel('Number of universities')

plt.show()

In [ ]:
# seems like there are 4 universities that haven't been delivering the Medicine career since 2010, let's check which ones they are
bool_2010 = {'since_2010': dfs_unified.groupby('university')['university'].count() == 12}
bool_2010_df = pd.DataFrame(bool_2010, columns=['since_2010'])
bool_2010_df[bool_2010_df['since_2010'] == False]

In [ ]:
year_uni_bb = dfs_unified[dfs_unified['university'] == 'universidad de barcelona-bellvitge']['year'].min()
year_uni_ib = dfs_unified[dfs_unified['university'] == 'universidad de las islas baleares']['year'].min()
year_uni_ji = dfs_unified[dfs_unified['university'] == 'universidad jaume i']['year'].min()
year_uni_pn = dfs_unified[dfs_unified['university'] == 'universidad pública de navarra']['year'].min()

In [ ]:
# we create a function tso that we can apply it later to the predicted 2022 data
def since_2010():
    dfs_unified['since_2010'] = ''
    dfs_unified['since_2010'] = np.where(dfs_unified['university'] == 'universidad de barcelona-bellvitge', year_uni_bb, dfs_unified['since_2010'])
    dfs_unified['since_2010'] = np.where(dfs_unified['university'] == 'universidad de las islas baleares', year_uni_ib, dfs_unified['since_2010'])
    dfs_unified['since_2010'] = np.where(dfs_unified['university'] == 'universidad jaume i', year_uni_ji, dfs_unified['since_2010'])
    dfs_unified['since_2010'] = np.where(dfs_unified['university'] == 'universidad pública de navarra', year_uni_pn, dfs_unified['since_2010'])
    dfs_unified['since_2010'] = np.where(dfs_unified['since_2010'] == '', True, dfs_unified['since_2010'])

In [ ]:
since_2010()

In [ ]:
dfs_unified['since_2010'].nunique()

In [ ]:
dfs_unified[dfs_unified['since_2010'] != True]

## Convert columns with numbers to floats

In [ ]:
# to convert to float, we can't have commas or spaces, as this function '.astype(float)' will give back an error

In [ ]:
# float to 2 decimals?? some have 3

In [ ]:
#dfs_unified['1_list'] = dfs_unified['1_list'].astype(float)

# Feature Engineering

## Create City and CCAA columns

### City

In [ ]:
# first: bigger cities in case two names appear
list_of_cities = ['madrid', 'zaragoza', 'barcelona', 'sevilla', 'valencia', 'murcia', 'granada','málaga', 'córdoba', 'cádiz', 'alicante', 'albacete', \
    'valladolid', 'badajoz', 'gerona', 'lérida', 'reus', 'navarra', 'santander', 'oviedo', \
    'santiago de compostela', 'salamanca', 'bilbao', 'salamanca', 'navarra', 'las palmas', 'la laguna', 'alcalá', 'ciudad real', 'huesca']

In [ ]:
num_cities = len(list_of_cities)

In [ ]:
# we create a function so that we can apply it later to the predicted 2022 data

def city():
    i=0
    dfs_unified['city'] = ''
    while i < num_cities:
        dfs_unified['city'] = np.where(dfs_unified['university'].str.contains(list_of_cities[i], na=False), list_of_cities[i], dfs_unified['city'])
        i +=1
    
    dfs_unified['city'] = np.where(dfs_unified['university'].str.contains('país vasco', na=False), 'bilbao', dfs_unified['city'])
    dfs_unified['city'] = np.where(dfs_unified['university'].str.contains('alcorcón', na=False), 'madrid', dfs_unified['city'])
    dfs_unified['city'] = np.where(dfs_unified['university'].str.contains('c.real', na=False), 'ciudad real', dfs_unified['city'])
    dfs_unified['city'] = np.where(dfs_unified['university'].str.contains('jaume', na=False), 'castellon de la plana', dfs_unified['city'])
    dfs_unified['city'] = np.where(dfs_unified['university'].str.contains('islas baleares', na=False), 'palma de mallorca', dfs_unified['city'])

    num_columns = len(dfs_unified_columns)

    while i < num_columns:
        dfs_unified[dfs_unified_columns[i]].replace({'': np.nan}, inplace=True)
        i += 1

In [ ]:
city()

In [ ]:
dfs_unified[dfs_unified['city'].isnull()]

### CCAA - Communities of Spain

In [ ]:
# CCAA - communities
def CCAA():
    dfs_unified['CCAA'] = ''
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['huesca', 'zaragoza']), 'Aragón', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['lérida', 'gerona', 'reus', 'barcelona']), 'Cataluña', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['madrid', 'alcalá']), 'C. Madrid', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['valladolid', 'salamanca']), 'Castilla y León', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['santander']), 'Cantabria', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['santiago de compostela']), 'Galicia', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['badajoz']), 'Extremadura', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['cádiz', 'córdoba', 'sevilla', 'málaga', 'granada']), 'Andalucía', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['murcia']), 'Murcia', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['ciudad real', 'albacete']), 'Castilla-La Mancha', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['bilbao']), 'País Vasco', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['la laguna', 'las palmas']), 'Canarias', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['valencia', 'castellon de la plana', 'alicante']), 'C. Valenciana', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['oviedo']), 'Asturias', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['palma de mallorca']), 'Islas Baleares', dfs_unified['CCAA'])
    dfs_unified['CCAA'] = np.where(dfs_unified['city'].isin(['navarra']), 'Navarra', dfs_unified['CCAA'])

In [ ]:
CCAA()

In [ ]:
dfs_unified[dfs_unified['CCAA'] == '']

In [ ]:
dfs_unified['CCAA'].unique()

In [ ]:
# CCAA table for streamlit visualization
# df_CCAA = dfs_unified.groupby('CCAA')['final_grade'].mean().reset_index()
# df_CCAA

## Difference of 1_list and final_grade scores between years

In [ ]:
def diff_between_years(field_name, score):
    dfs_unified[field_name] = ''

    # year 2010 (initial year)
    for uni in list_of_uni_names_2021:
        dfs_unified[field_name] = np.where((dfs_unified['university'] == uni) & (dfs_unified['year'] == 2010), np.nan, dfs_unified[field_name])
    # rest of years
    for uni in list_of_uni_names_2021:
        for year in range(2011, 2023):
            try:
                dfs_unified[field_name] = np.where((dfs_unified['university'] == uni) & (dfs_unified['year'] == year), float((dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] == year)][score]).values[0] - (dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] == year - 1)][score]).values[0]), dfs_unified[field_name])
            except:
                pass

    # for the 4 universities that don't have data since 2010
    dfs_unified[field_name].replace({'': np.nan}, inplace=True)

In [ ]:
diff_between_years('diff_1_list', '1_list')

In [ ]:
diff_between_years('diff_final_grade', 'final_grade')

## Decreased value between 1_list and final_grade

In [ ]:
# decresead value from 1_list to final grade
def diff_1_list_final_grade():
    dfs_unified['diff_1_list_final_grade'] = dfs_unified['1_list'] - dfs_unified['final_grade']

In [ ]:
diff_1_list_final_grade()

## Covid format change: change in test format due to covid

In [ ]:
# compare 1_list growth of years before 2020, and in 2020
grouped_year_1_list = dfs_unified.groupby('year')['diff_1_list'].mean().reset_index()

# round to 3 decimals
grouped_year_1_list['diff_1_list'] = grouped_year_1_list['diff_1_list'].apply(lambda x: round(x, 3))

# since we don't have data from 2009, we are excluding 2010
# we are also excluding 2021 since it's not relevant to show the growth before and after covid
grouped_year_1_list = grouped_year_1_list[(grouped_year_1_list['year'] != 2010) & (grouped_year_1_list['year'] != 2021)]

grouped_year_1_list_before_2020 = grouped_year_1_list[grouped_year_1_list['year'] < 2020]
grouped_year_1_list_in_2020 = grouped_year_1_list[grouped_year_1_list['year'] == 2020]


fig,ax=plt.subplots(figsize=(12,8))
bars = ax.bar(grouped_year_1_list['year'], grouped_year_1_list['diff_1_list'])
# show number on top of bars
ax.bar_label(bars)
# style
plt.style.use('ggplot')
# title and axis
plt.xticks(range(min(grouped_year_1_list['year']), max(grouped_year_1_list['year']) + 1))
plt.title('Average difference of 1_list scores across years')
plt.xlabel('Year')
plt.ylabel('Avg diff of 1_list scores between curr and prev year')

plt.show()


print('average 1_list difference before 2020 is: ', grouped_year_1_list_before_2020['diff_1_list'].mean())
print('average 1_list difference in 2020 is: ',grouped_year_1_list_in_2020['diff_1_list'].mean())
print('The difference between average before 2020 and average in 2020 is x',grouped_year_1_list_in_2020['diff_1_list'].mean() / grouped_year_1_list_before_2020['diff_1_list'].mean())

In [ ]:
# change in test format due to covid

def covid_format_change():
    dfs_unified['covid_format_change'] = ''
    dfs_unified['covid_format_change'] = np.where((dfs_unified['year'] >= 2020), float(1), dfs_unified['covid_format_change'])
    dfs_unified['covid_format_change'] = np.where((dfs_unified['year'] < 2020), float(0), dfs_unified['covid_format_change'])

In [ ]:
covid_format_change()

## Round columns above to 3 decimals

In [ ]:
# round numbers to 3 decimals
def round_to_3():
    dfs_unified['1_list'] = dfs_unified['1_list'].apply(lambda x: round(x, 3))
    dfs_unified['final_grade'] = dfs_unified['final_grade'].apply(lambda x: round(x, 3))
    dfs_unified['diff_final_grade'] = dfs_unified['diff_final_grade'].apply(lambda x: round(x, 3))
    dfs_unified['diff_1_list'] = dfs_unified['diff_1_list'].apply(lambda x: round(x, 3))
    dfs_unified['diff_1_list_final_grade'] = dfs_unified['diff_1_list_final_grade'].apply(lambda x: round(x, 3))

In [ ]:
round_to_3()

## Shorter name of universities

In [ ]:
dfs_unified['university'] = dfs_unified['university'].replace('universidad', 'u.', regex=True)
list_of_uni_names_2021  = dfs_unified[dfs_unified['year'] == last_year]['university'].tolist()

## Latitude and Longitude of Universities for map on Streamlit

In [ ]:
#https://www.mapcoordinates.net/es

def latitude():
    dfs_unified['latitude'] = ''
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. autónoma de barcelona', 41.5025932, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de lérida', 41.81636125, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de gerona', 41.9743792, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. rovira i virgili (reus)', 41.155309, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de zaragoza (campus huesca)', 42.1414059, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de valladolid', 41.6105937, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de cantabria (santander)', 43.4736983, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de santiago de compostela', 42.88050025, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de extremadura (badajoz)', 38.8824069, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de cádiz', 36.5340199, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. pompeu fabra (barcelona)', 41.37921475, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de barcelona-clínico', 41.39017434, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de salamanca', 40.9666432, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de zaragoza (campus zaragoza)', 41.6421312, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de c-la mancha (campus c.real)', 38.9918372, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de córdoba', 37.8560421, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. del país vasco (lejona)', 43.3359821, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de c-la mancha (campus albacete)', 38.4527362, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de sevilla', 37.4128557, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. miguel hernández (s. juan de alicante)', 38.39896277, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de alcalá', 40.4818396, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de málaga', 36.7200431, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de murcia', 37.99109245, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de la laguna', 28.4852475, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'urjc - campus alcorcón', 40.3506324, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de valencia', 39.4786642, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. autónoma de madrid', 40.4832261, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de las palmas', 28.09055808, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de granada', 37.1474408, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. complutense de madrid', 40.4437398, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de oviedo', 43.3533657, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. jaume i', 39.9899483, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de las islas baleares', 39.60701435, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. pública de navarra', 42.8154264, dfs_unified['latitude'])
    dfs_unified['latitude'] = np.where(dfs_unified['university'] == 'u. de barcelona-bellvitge', 41.3507816, dfs_unified['latitude'])

In [ ]:
def longitude():
    dfs_unified['longitude'] = ''
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. autónoma de barcelona', 2.080056, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de lérida', 1.59713745, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de gerona', 2.8235429, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. rovira i virgili (reus)', 1.1053273, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de zaragoza (campus huesca)', -0.4060896, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de valladolid', -4.6916619, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de cantabria (santander)', -3.7859174, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de santiago de compostela', -8.5457602, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de extremadura (badajoz)', -7.0194457, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de cádiz', -6.3018987, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. pompeu fabra (barcelona)', 2.17941527, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de barcelona-clínico', 2.15212588, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de salamanca', -5.6772757, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de zaragoza (campus zaragoza)', -0.90302271, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de c-la mancha (campus c.real)', -3.9268263, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de córdoba', -4.8134452, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. del país vasco (lejona)', -2.9768285, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de c-la mancha (campus albacete)', -2.05363374, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de sevilla', -5.9845606, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. miguel hernández (s. juan de alicante)', -0.43477806, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de alcalá', -3.3644973, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de málaga', -4.41617939, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de murcia', -1.13010405, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de la laguna', -16.3126059, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'urjc - campus alcorcón', -3.8437926, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de valencia', -0.3627246, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. autónoma de madrid', -3.6972262, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de las palmas', -15.41890014, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de granada', -3.6042349, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. complutense de madrid', -3.72614639, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de oviedo', -5.8687712, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. jaume i', -0.0664218, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de las islas baleares', 2.64475448, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. pública de navarra', -1.65263107, dfs_unified['longitude'])
    dfs_unified['longitude'] = np.where(dfs_unified['university'] == 'u. de barcelona-bellvitge', 2.1045067, dfs_unified['longitude'])

In [ ]:
latitude()
longitude()

In [ ]:
dfs_unified.head()

In [ ]:
dfs_unified[dfs_unified['latitude'] == '']

In [ ]:
dfs_unified[dfs_unified['longitude'] == '']

In [ ]:
dfs_unified[dfs_unified['latitude'].isnull()]

In [ ]:
dfs_unified[dfs_unified['longitude'].isnull()]

# Model

In [ ]:
dfs_unified.head()

In [ ]:
dfs_unified_columns

In [ ]:
dfs_unified_columns = dfs_unified.columns.tolist()[1:]

num_columns = len(dfs_unified_columns)

for i in range(num_columns):
    dfs_unified[dfs_unified_columns[i]].replace({np.nan: 0}, inplace=True)

In [ ]:
#ojo que paso los nan a 0

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))         # Sample figsize in inches
sns.heatmap(dfs_unified.corr(), annot=True, fmt=".2f", linewidths=.5, ax=ax, cmap='RdBu')
plt.title('Correlation between variables');

## Create and evaluate models

As the output variable (1_list) is a number we need to solve a regression problem.

### Linear Regression

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
def plot_actual_vs_predicted(title_string, subtitle_string, y_test, y_predict, year):
    # visualize the predicted score as a line on the test set
    plt.figure(figsize=(12,6))
    plt.scatter(dfs_unified[dfs_unified['year'] == year]['university'], y_test, color = 'red', label = 'actual')
    plt.plot(dfs_unified[dfs_unified['year'] == year]['university'], y_predict, color='g', label = 'predicted')
    plt.xlabel('University', fontsize = 14)
    plt.ylabel('1_list', fontsize = 14)
    plt.xticks(rotation=90)
    plt.suptitle(title_string, y=1.05, fontsize=18)
    plt.title(subtitle_string, fontsize=16)
    plt.legend()
    plt.style.use('ggplot')
    plt.show()

In [ ]:
results_models = pd.DataFrame(columns=['model_name_variables', 'MSE', 'RMSE', 'MAE'])

In [ ]:
def plot_metrics():
    plt.style.use('ggplot')

    df_MSE = results_models[['MSE']]
    df_RMSE = results_models[['RMSE']]
    df_MAE = results_models[['MAE']]

    # sort values so the best model appears in first position
    df_MSE = df_MSE.sort_values('MSE', ascending=False)
    df_RMSE = df_RMSE.sort_values('RMSE', ascending=False)
    df_MAE = df_MAE.sort_values('MAE', ascending=False)

    # create figure and one axis per metric
    f = plt.figure(figsize=(10,20))
    axes1 = f.add_subplot(4,1,1)
    axes2 = f.add_subplot(4,1,2)
    axes3 = f.add_subplot(4,1,3)
        
    # plot dataframes with DataFrame.plot()
    df_MSE.plot(kind='barh', ax=axes1, color='lightgreen', title = 'Compare Model Metrics')\
            .legend(loc='lower left',bbox_to_anchor=(1.0, 0.5))
    df_RMSE.plot(kind='barh', ax=axes2, color='lightskyblue')\
            .legend(loc='lower left',bbox_to_anchor=(1.0, 0.5))
    df_MAE.plot(kind='barh',ax=axes3, color='salmon')\
            .legend(loc='lower left',bbox_to_anchor=(1.0, 0.5))  
        
    # annotate bars with result values 
    for p in axes1.patches:
            axes1.annotate(np.round(a=p.get_width(), decimals=3), \
                    (p.get_x() + p.get_width(), p.get_y()), \
                    xytext=(-30, 5), textcoords='offset points')

    for p in axes2.patches:
            axes2.annotate(np.round(a=p.get_width(), decimals=3), \
                    (p.get_x() + p.get_width(), p.get_y()), \
                    xytext=(-30, 5), textcoords='offset points')

    for p in axes3.patches:
            axes3.annotate(np.round(a=p.get_width(), decimals=3), \
                    (p.get_x() + p.get_width(), p.get_y()), \
                    xytext=(-30, 5), textcoords='offset points')
    plt.show()

In [ ]:
# Train data will be years 2011-2020, Test data will be year 2021, and Validation data year 2010

In [ ]:
def linear_regression(X_var, model_name_variables):
    lr = LinearRegression()

    X_train = dfs_unified[(dfs_unified['year'] != 2021) & (dfs_unified['year'] != 2011)][X_var]
    y_train = dfs_unified[(dfs_unified['year'] != 2021) & (dfs_unified['year'] != 2011)][['1_list']]

    X_test = dfs_unified[(dfs_unified['year'] == 2021)][X_var]
    y_test = dfs_unified[(dfs_unified['year'] == 2021)][['1_list']]

    lr.fit(X_train, y_train)

    y_predict = lr.predict(X_test) # we are predicting the 1_list scores for 2021, for all of the universities (various scores)
    
    
    # evaluation
    errors = y_test - y_predict

    # MSE (min squared error)
    mse = float(np.mean(errors ** 2))

    # RMSE (root min squared error)
    rmse = float(np.sqrt(mse))

    # MAE (mean absolute error)
    mae = float(np.mean(np.abs(errors)))
    
    results_models.loc[model_name_variables] = [model_name_variables, mse, rmse, mae]

    plot_actual_vs_predicted('Linear Regression: For 2021, actual 1_list scores vs predicted', f'Variables: {X_var}', y_test, y_predict, 2021)

    

In [ ]:
# using only "final_grade" variable

linear_regression(['final_grade'], 'linear_regression_final_grade')

In [ ]:
# using "final_grade" and "year" variables
linear_regression(['final_grade', 'year'], 'linear_regression_final_grade_year')

In [ ]:
# using "final_grade", "year" and "covid_format_change" variables
linear_regression(['final_grade', 'year', 'covid_format_change'], 'linear_regression_final_grade_year_covid')

In [ ]:
# using "final_grade", "year", "covid_format_change" and "diff_1_list"  variables
linear_regression(['final_grade', 'year', 'covid_format_change', 'diff_1_list'], 'linear_regression_final_grade_year_covid_diff_1_list')

In [ ]:
plot_metrics()

### Linear Regression per University

In [ ]:
# evaluate models per university
results_models_per_uni = pd.DataFrame(columns=['university','MSE', 'RMSE', 'MAE', 'y_predict', 'y_test'])
results_models_per_uni['university'] = list_of_uni_names_2021

In [ ]:
def linear_regression_per_uni(X_var, model_name_variables):
    lr = LinearRegression()
    X_train = dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] != 2021) & (dfs_unified['year'] != 2011)][X_var]
    y_train = dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] != 2021) & (dfs_unified['year'] != 2011)][['1_list']]

    X_test = dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] == 2021)][X_var]
    y_test = dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] == 2021)][['1_list']]

    lr.fit(X_train, y_train)

    y_predict = lr.predict(X_test) # we are predicting the 1_list score from 2021 for a specific university (only 1 score)
    y_predict = float(y_predict)
    y_test = y_test['1_list'].values[0]

    # evaluation
    errors = y_test - y_predict

    # MSE (min squared error)
    mse = np.mean(errors ** 2)

    # RMSE (root min squared error)
    rmse = np.sqrt(mse)

    # MAE (mean absolute error)
    mae = np.mean(np.abs(errors))
    
    results_models_per_uni['MSE'] = np.where(results_models_per_uni['university'] == uni, mse, results_models_per_uni['MSE'])
    results_models_per_uni['RMSE'] = np.where(results_models_per_uni['university'] == uni, rmse, results_models_per_uni['RMSE'])
    results_models_per_uni['MAE'] = np.where(results_models_per_uni['university'] == uni, mae, results_models_per_uni['MAE'])
    results_models_per_uni['y_predict'] = np.where(results_models_per_uni['university'] == uni, y_predict, results_models_per_uni['y_predict'])
    results_models_per_uni['y_test'] = np.where(results_models_per_uni['university'] == uni, y_test, results_models_per_uni['y_test'])

    # we do the average out of all of the universities, to compare against the other models
    results_models.loc[model_name_variables] = [model_name_variables, results_models_per_uni['MSE'].mean(), results_models_per_uni['RMSE'].mean(), results_models_per_uni['MAE'].mean()]

In [ ]:
# using only "final_grade" variable

X_var = ['final_grade']

for uni in list_of_uni_names_2021:
    linear_regression_per_uni(X_var, 'linear_regression_per_uni_final_grade')

y_test = results_models_per_uni['y_test']
y_predict = results_models_per_uni['y_predict']

plot_actual_vs_predicted('Linear Regression Per University: For 2021, actual 1_list scores vs predicted', f'Variables: {X_var}', y_test, y_predict, 2021)

In [ ]:
# using "final_grade" and "year" variables

X_var = ['final_grade', 'year']

for uni in list_of_uni_names_2021:
    linear_regression_per_uni(X_var, 'linear_regression_per_uni_final_grade_year')

y_test = results_models_per_uni['y_test']
y_predict = results_models_per_uni['y_predict']

plot_actual_vs_predicted('Linear Regression Per University: For 2021, actual 1_list scores vs predicted', f'Variables: {X_var}', y_test, y_predict, 2021)

In [ ]:
# using "final_grade", "year" and "covid_format_change" variables

X_var = ['final_grade', 'year', 'covid_format_change']

for uni in list_of_uni_names_2021:
    linear_regression_per_uni(X_var, 'linear_regression_per_uni_final_grade_year_covid')

y_test = results_models_per_uni['y_test']
y_predict = results_models_per_uni['y_predict']

plot_actual_vs_predicted('Linear Regression Per University: For 2021, actual 1_list scores vs predicted', f'Variables: {X_var}', y_test, y_predict, 2021)

In [ ]:
# using "final_grade", "year", "covid_format_change" and "diff_1_list"  variables

X_var = ['final_grade', 'year', 'covid_format_change', 'diff_1_list']

for uni in list_of_uni_names_2021:
    linear_regression_per_uni(X_var, 'linear_regression_per_uni_final_grade_year_covid_diff_1_list')

y_test = results_models_per_uni['y_test']
y_predict = results_models_per_uni['y_predict']

plot_actual_vs_predicted('Linear Regression Per University: For 2021, actual 1_list scores vs predicted', f'Variables: {X_var}', y_test, y_predict, 2021)

In [ ]:
plot_metrics()

In [ ]:
# given that the metrics are quite below the mean score of the variable we are predicting, we can consider the models are efficient

dfs_unified['1_list'].mean()

### ARIMA per University

In [ ]:
#!pip install pmdarima --upgrade
#!pip install statsmodels==0.11.0rc1

In [ ]:
from statsmodels.tsa.stattools import adfuller
#import pandas.util.testing as tm

def ad_test(dataset):
    dftest = adfuller(dataset, autolag = 'AIC')
    print('1. ADF: ', dftest[0])
    print('2. P-Value: ', dftest[1])
    print('3. Num Of Lags: ', dftest[2])
    print('4. Num Of Observations Used For ADF Regression and Critical Values Calculation: ', dftest[3])
    print('5. Critical Values: ')
    for key, val in dftest[4].items():
        print('\t', key, ': ', val)


In [ ]:
ad_test(dfs_unified['1_list'])

In [ ]:
from pmdarima import auto_arima
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
#from statsmodels.tsa.arima.model import ARIMA

In [ ]:
# split data in train and test
university = 'u. complutense de madrid'
train = dfs_unified[(dfs_unified['university'] == university) & (dfs_unified['year'] != 2021) & (dfs_unified['year'] != 2011)][['1_list']]
test = dfs_unified[(dfs_unified['university'] == university) & (dfs_unified['year'] == 2021)][['1_list']]

#print(dfs_unified.shape)
print(train.shape, test.shape)

In [ ]:
dfs_unified[(dfs_unified['university'] == university)]

In [ ]:
def ARIMACheck(university):
    from pmdarima import auto_arima  

    stepwise_fit = auto_arima(dfs_unified[dfs_unified['university'] == university]['1_list'], trace=True, suppress_warnings=True)
    #print(stepwise_fit.summary())
    return stepwise_fit

In [ ]:
stepwise_fit = ARIMACheck(university)
stepwise_fit

In [ ]:
stepwise_fit.get_params().get('order')

In [ ]:
model = ARIMA(train, order=stepwise_fit.get_params().get('order'))
model = model.fit()
model.summary()

In [ ]:
# prediction for test set (2021 scores)
start = len(train)
end = len(train) + len(test)-1
y_predict = model.predict(start=start, end=end, typ='levels')
y_predict.index = dfs_unified[(dfs_unified['university'] == university) & (dfs_unified['year'] == 2021)]['university']
y_predict_df = y_predict.reset_index()
y_predict_df['1_list'] = ''
y_predict_df['1_list'] = y_predict_df[0]
y_predict_df = y_predict_df.drop(0, 1)
y_predict_df

In [ ]:
test['1_list'].values[0]

In [ ]:
train['1_list'].values[0]

In [ ]:
y_predict.values[0]

In [ ]:
# evaluate models per university
arima_results_models_per_uni = pd.DataFrame(columns=['university','MSE', 'RMSE', 'MAE', 'y_predict', 'y_test'])
arima_results_models_per_uni['university'] = list_of_uni_names_2021

In [ ]:
def ARIMA_per_uni(model_name_variables):
    from statsmodels.tsa.arima.model import ARIMA
    
    # split data in train and test
    train = dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] != 2021) & (dfs_unified['year'] != 2011)][['1_list']]
    test = dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] == 2021)][['1_list']]

    stepwise_fit = ARIMACheck(uni)

    model = ARIMA(train, order=stepwise_fit.get_params().get('order'))
    model = model.fit()
    
    # prediction for test set (2021 scores)
    start = len(train)
    end = len(train) + len(test)-1
    y_predict = model.predict(start=start, end=end, typ='levels')
    # y_predict.index = dfs_unified[(dfs_unified['university'] == university) & (dfs_unified['year'] == 2021)]['university']
    # y_predict_df['university'] = np.where(y_predict_df['university'] == university, university, y_predict_df['university'])
    # y_predict_df['1_list'] = np.where(y_predict_df['university'] == university, pred.mean(), y_predict_df['1_list'])
    
    y_predict = y_predict.values[0]
    y_test = test['1_list'].values[0]

    # evaluation
    errors = y_test - y_predict

    # MSE (min squared error)
    mse = np.mean(errors ** 2)

    # RMSE (root min squared error)
    rmse = np.sqrt(mse)

    # MAE (mean absolute error)
    mae = np.mean(np.abs(errors))
    
    arima_results_models_per_uni['MSE'] = np.where(arima_results_models_per_uni['university'] == uni, mse, arima_results_models_per_uni['MSE'])
    arima_results_models_per_uni['RMSE'] = np.where(arima_results_models_per_uni['university'] == uni, rmse, arima_results_models_per_uni['RMSE'])
    arima_results_models_per_uni['MAE'] = np.where(arima_results_models_per_uni['university'] == uni, mae, arima_results_models_per_uni['MAE'])
    arima_results_models_per_uni['y_predict'] = np.where(arima_results_models_per_uni['university'] == uni, y_predict, arima_results_models_per_uni['y_predict'])
    arima_results_models_per_uni['y_test'] = np.where(arima_results_models_per_uni['university'] == uni, y_test, arima_results_models_per_uni['y_test'])

    # we do the average out of all of the universities, to compare against the other models
    results_models.loc[model_name_variables] = [model_name_variables, arima_results_models_per_uni['MSE'].mean(), arima_results_models_per_uni['RMSE'].mean(), arima_results_models_per_uni['MAE'].mean()]

In [ ]:
#list_of_uni_names_2021

In [ ]:
#!pip install statsmodels --upgrade

In [ ]:
for uni in list_of_uni_names_2021:
    print(uni)
    try:
        ARIMA_per_uni('ARIMA_per_uni')
    except:
        pass

In [ ]:
arima_results_models_per_uni

In [ ]:
dfs_unified[(dfs_unified['university'] == 'u. de barcelona-bellvitge') & (dfs_unified['year'] != 2021)][['1_list']].mean()

In [ ]:
# for u. de barcelona-bellvitge we only have data since 2020, therefore we get an error when trying to calculate the scores
# for this case, let's suppose there is no growth (same score for 2021 as 2020)

uni = 'u. de barcelona-bellvitge'
y_predict = dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] != 2021)][['1_list']].mean()
y_test = dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] == 2021)][['1_list']].mean()

# evaluation
errors = y_test - y_predict

# MSE (min squared error)
mse = np.mean(errors ** 2)

# RMSE (root min squared error)
rmse = np.sqrt(mse)

# MAE (mean absolute error)
mae = np.mean(np.abs(errors))
    
arima_results_models_per_uni['MSE'] = np.where(arima_results_models_per_uni['university'] == uni, mse, arima_results_models_per_uni['MSE'])
arima_results_models_per_uni['RMSE'] = np.where(arima_results_models_per_uni['university'] == uni, rmse, arima_results_models_per_uni['RMSE'])
arima_results_models_per_uni['MAE'] = np.where(arima_results_models_per_uni['university'] == uni, mae, arima_results_models_per_uni['MAE'])
arima_results_models_per_uni['y_predict'] = np.where(arima_results_models_per_uni['university'] == uni, y_predict, arima_results_models_per_uni['y_predict'])
arima_results_models_per_uni['y_test'] = np.where(arima_results_models_per_uni['university'] == uni, y_test, arima_results_models_per_uni['y_test'])

    # we do the average out of all of the universities, to compare against the other models
results_models.loc['ARIMA_per_uni'] = ['ARIMA_per_uni', arima_results_models_per_uni['MSE'].mean(), arima_results_models_per_uni['RMSE'].mean(), arima_results_models_per_uni['MAE'].mean()]

In [ ]:
arima_results_models_per_uni

In [ ]:
X_var = ['1_list']
plot_actual_vs_predicted('ARIMA Per University: For 2021, actual 1_list scores vs predicted', f'Variables: {X_var}', arima_results_models_per_uni['y_test'], arima_results_models_per_uni['y_predict'], 2021)

In [ ]:
plot_metrics()

## Check if there is Overfitting

We are going to check two things:
- graph of predicted vs actual 1_list values for validation data (year: 2011)
- plot and compare metrics of the validation data and the test data, they should be similar

In [ ]:
list_of_uni_names_2011 = dfs_unified[dfs_unified['year'] == 2011]['university'].tolist()

In [ ]:
# evaluate models per university
results_models_per_uni = pd.DataFrame(columns=['university','MSE', 'RMSE', 'MAE', 'y_predict', 'y_val'])
results_models_per_uni['university'] = list_of_uni_names_2011

In [ ]:
def linear_regression_per_uni(X_var, model_name_variables):
    lr = LinearRegression()
    X_train = dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] != 2021) & (dfs_unified['year'] != 2011)][X_var]
    y_train = dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] != 2021) & (dfs_unified['year'] != 2011)][['1_list']]

    X_val = dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] == 2011)][X_var]
    y_val = dfs_unified[(dfs_unified['university'] == uni) & (dfs_unified['year'] == 2011)][['1_list']]

    lr.fit(X_train, y_train)

    y_predict = lr.predict(X_val) # we are predicting the 1_list score from 2021 for a specific university (only 1 score)
    y_predict = float(y_predict)
    y_val = y_val['1_list'].values[0]

    # evaluation
    errors = y_val - y_predict

    # MSE (min squared error)
    mse = np.mean(errors ** 2)

    # RMSE (root min squared error)
    rmse = np.sqrt(mse)

    # MAE (mean absolute error)
    mae = np.mean(np.abs(errors))
    
    results_models_per_uni['MSE'] = np.where(results_models_per_uni['university'] == uni, mse, results_models_per_uni['MSE'])
    results_models_per_uni['RMSE'] = np.where(results_models_per_uni['university'] == uni, rmse, results_models_per_uni['RMSE'])
    results_models_per_uni['MAE'] = np.where(results_models_per_uni['university'] == uni, mae, results_models_per_uni['MAE'])
    results_models_per_uni['y_predict'] = np.where(results_models_per_uni['university'] == uni, y_predict, results_models_per_uni['y_predict'])
    results_models_per_uni['y_val'] = np.where(results_models_per_uni['university'] == uni, y_val, results_models_per_uni['y_val'])

    # we do the average out of all of the universities, to compare against the other models
    results_models.loc[model_name_variables] = [model_name_variables, results_models_per_uni['MSE'].mean(), results_models_per_uni['RMSE'].mean(), results_models_per_uni['MAE'].mean()]

In [ ]:
# using "final_grade", "year" and "covid_format_change" variables

X_var = ['final_grade', 'year', 'covid_format_change']

for uni in list_of_uni_names_2011:
    linear_regression_per_uni(X_var, 'linear_regression_per_uni_final_grade_year_covid_VALIDATION')

y_val = results_models_per_uni['y_val']
y_predict = results_models_per_uni['y_predict']

plot_actual_vs_predicted('Linear Regression Per University: For 2011, actual 1_list scores vs predicted', f'Variables: {X_var}', y_val, y_predict, 2011)

In [ ]:
plot_metrics()

## Predict 2022 1_list scores

Given the metrics results shown above, we will use the Linear Regression per University models with the variables: "final_grade", "year" and "covid_format_change" to predict the scores for 2022

In [ ]:
# now that we now the model is good, we retrain the model on the entire dataset (not only the train section)

In [ ]:
pred_y_df = pd.DataFrame(columns=['year', 'university', '1_list'])
pred_y_df['university'] = list_of_uni_names_2021

In [ ]:
def predict_and_save_scores(year, covid_format_change, university):
    '''
    explicar
    '''
    lr = LinearRegression()
    X = dfs_unified[dfs_unified['university'] == university][['final_grade', 'year', 'covid_format_change']]
    y = dfs_unified[dfs_unified['university'] == university][['1_list']]

    # train
    lr.fit(X, y)
    
    # predict, supposing final_grade as previous year's final_grade
    final_grade = dfs_unified[dfs_unified['university'] == university]['final_grade'][-1:].mean()
    
    pred_y_df['1_list'] = np.where(pred_y_df['university'] == university, float(lr.predict([[final_grade, year, covid_format_change]])), pred_y_df['1_list'])

In [ ]:
for uni in list_of_uni_names_2021:
    predict_and_save_scores(2022, 1.0, uni)

In [ ]:
# problema: 1_list no puede ser menor que final_grade | lo calculamos sin utilizar el modelo, cogemos el decrecimiento del último año y se lo restamos a 1_list predicted
pred_y_df['final_grade'] = ''

for uni in list_of_uni_names_2021:
    pred_y_df['final_grade'] = np.where(pred_y_df['university'] == uni, pred_y_df['1_list'] - dfs_unified[dfs_unified['university'] == uni]['diff_1_list_final_grade'][-1:].mean(), pred_y_df['final_grade'])

In [ ]:
pred_y_df['year'] = 2022

In [ ]:
pred_y_df

In [ ]:
# append predicted scores to dfs_unified

dfs_unified = dfs_unified.append(pred_y_df)

In [ ]:
dfs_unified

In [ ]:
# apply created columns to predicted data (2022)
since_2010()
city()
CCAA()
covid_format_change()
diff_between_years('diff_1_list', '1_list')
diff_between_years('diff_final_grade', 'final_grade')
diff_1_list_final_grade()
round_to_3()
latitude()
longitude()

In [ ]:
dfs_unified[dfs_unified['year'] == 2022]

# Export data

In [ ]:
exported_data_directory = your_directory + '/output'
exported_data_directory

In [ ]:
# export data to be used on streamlit
file_name = exported_data_directory + '/exported_data_notebook.csv'
dfs_unified.to_csv(file_name, index = False, header=True)

In [ ]:
dfs_unified.notnull()

In [ ]:
dfs_unified[dfs_unified.isnull()]


In [ ]:
dfs_unified[dfs_unified['university'] == '']

# Run Front-end / Streamlit visualization (myapp.py)

In [ ]:
# execute cell below

In [ ]:
#!pip install streamlit
#!streamlit run /Users/miguel/repos/Prediction_Medicine_Selectivity_Scores/streamlit/myapp.py